## Tobigs 2주차 Regression 과제
#### 15기 이윤정

---
## Matrix 구현 
1. Linear Regression 에서 쓰이는 **정규방정식**을 행렬로 구현하고, sklearn 혹은 OLS 패키지를 통해 구한 **실제 값과 비교**해 주세요 
2. LSE에서 쓰이는 Loss Function, **MSE**를 행렬로 구현해 출력해 주세요 

### Data Load 

In [1]:
import pandas as pd 
data = pd.read_csv("assignment1.csv")

In [2]:
data.head()

,y,x1,x2,x3,x4,x5
0,10,38.9,64.7,4,868,59.7
1,13,41.6,45.3,-4,957,61.4
2,11,39.7,74.1,8,786,61.0
3,7,37.3,48.0,19,984,67.5
4,10,39.5,51.9,6,700,57.2


In [3]:
data.shape # y:(9,1), X:(9,5)

(9, 6)

In [4]:
X = data.drop(["y"], axis=1)
y = data.y

In [5]:
# matrix 계산을 위해 numpy 형태로 바꾸어 줍니다 
X = X.to_numpy()
y = y.to_numpy()

In [6]:
X

array([[  38.9,   64.7,    4. ,  868. ,   59.7],
       [  41.6,   45.3,   -4. ,  957. ,   61.4],
       [  39.7,   74.1,    8. ,  786. ,   61. ],
       [  37.3,   48. ,   19. ,  984. ,   67.5],
       [  39.5,   51.9,    6. ,  700. ,   57.2],
       [  37.4,   53.6,   -5. , 1037. ,   58.8],
       [  35.1,   71.4,    3. ,  986. ,   58.6],
       [  38.8,   58.3,    6. ,  819. ,   59.2],
       [  36.6,   52.6,  -19. ,  791. ,   54.4]])

In [7]:
y

array([10, 13, 11,  7, 10,  9,  9,  6,  5], dtype=int64)

### 1. Normal Equation

정규방정식 $$\Theta = \left ( X^{T}X \right )^{-1}X^{T}y$$

In [8]:
import numpy as np
from numpy.linalg import inv 

In [9]:
def estimate_beta(X, y):
    designX = np.c_[np.ones((9,1)), X]
    beta_hat = np.linalg.inv(designX.T.dot(designX)).dot(designX.T).dot(y)
    
    return beta_hat

In [10]:
beta_hat = estimate_beta(X, y)
beta_hat

array([-3.92447368e+01,  1.31232583e+00,  8.53744361e-02,  7.41849897e-02,
        1.50018573e-02, -3.42273652e-01])

#### 실제 값과 비교

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
model = LinearRegression()
model.fit(X, y)

pred = model.predict(X)

In [13]:
print(model.intercept_) # intercaept
print(model.coef_) # 추정된 회귀계수 (intercept제외)

-39.24473678135658
[ 1.31232583  0.08537444  0.07418499  0.01500186 -0.34227365]


정규방정식을 통해 재현한 값과 유사하지만 정확하게 재현하지는 못하였다. 

### 2. MSE
$MSE = \frac{\sum (y - \widehat{y})^{2}}{n - k - 1}$, (n : 데이터 건수, k : 셜명변수 x의 수)

In [22]:
def MSE(X, y, beta_hat):
    designX = np.c_[np.ones((9,1)), X]
    y_pred = designX.dot(beta_hat)
    mse = sum(np.square(np.subtract(y,y_pred)))/(len(y)-len(X[0])-1)
    
    return mse

In [23]:
MSE(X, y, estimate_beta(X, y))

4.846793168705752

### 3.MLE

#### 바구니 안에 파란구슬과 빨간구슬이 들어있을 때, 10개를 추출하여보니 파란구슬이 2개 빨간구슬이 8개가 추출되었습니다. 
#### 이때 바구니안에는 파란구슬과 빨간구슬이 각각 몇개있는지에 대해서 최대우도추정법을 이용해 논리적으로 설명해주세요. 
- 수식은 있어도 좋고 말로 풀어쓰셔도 좋습니다
- 추출은 복원추출을 전제로 합니다
- 글자수 제한은 없습니다. 많이 쓰라는 얘기가 아니라 정말 제한이 없습니다. 너무 부담가지고 쓰지마세요.

바구니 안 총 구슬의 개수를 N, 바구니에서 구슬 10개를 추출할 때 나타날 수 있는 모든 사건을 E라고 한다면 바구니 내 파란 구슬의 비율 $p = \frac{파란 구슬 수}{N}$이다.  
각 시행은 독립이고, 복원추출이므로 각 사건의 확률 $P\left ( E_{k}|p \right )$는 각 확률의 곱으로 표현 할 수 있다. 따라서, 10개 추출 시 파란 구슬이 2개 빨간 구슬이 8개 추출될 확률은 $P\left ( E|p \right ) = \binom{10}{2}p^{2}(1-p)^{8}$이고, 관측된 사건의 확률을 최대화로 만들어주는 p의 추정량 $\hat{p} = argmax_{p}\binom{10}{2}p^{2}(1-p)^{8}$이다. $f(p) = P\left ( E_{k}|p \right ) = log{\binom{10}{2}p^{2}(1-p)^{8}}$이고, 최댓값을 찾기 위해 양변을 미분하면 $f'(p) = \frac{2}{p}-\frac{8}{1-p}$이다. 이때, $f'(p) = 0$에서 최댓값을 가지므로 $p=\hat{p}=\frac{2}{10}$이다. 앞서, $p = \frac{파란 구슬 수}{N}$로 정의되었으므로 바구니 속 파란 구슬의 개수는 $\frac{2*N}{10}$, 빨간 구슬의 개수는 $\frac{8*N}{10}$이다.